In [1]:
from data_gen_utils import *
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from synthetic.generate import SingleTaskTreeDepsGenerator
from metal.label_model import LabelModel, CliqueTree
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
M = 6
n = 10000
K = 1
dg = DataGenerator(M,n)

In [3]:
dg.generate_O(k=3)

In [4]:
print(dg.deps)

[(0, 1), (2, 3), (4, 5)]


In [5]:
# remove deps
print(dg.deps_no_diag)

[(0, 1), (2, 3), (4, 5)]


In [6]:
print(dg.L)

w,h = (dg.L).shape
L = np.zeros((w,h),dtype=float)
for i in range(w):
    for j in range(h):
        if dg.L[i,j] > 0:
            L[i,j] = 1
        else:
            L[i,j] = 0

[[ 1. -1. -1. ... -1. -1. -1.]
 [ 1. -1. -1. ...  1. -1. -1.]
 [ 1.  1.  1. ... -1. -1.  1.]
 [ 1. -1. -1. ... -1. -1. -1.]
 [ 1. -1. -1. ...  1. -1. -1.]
 [ 1. -1. -1. ... -1. -1. -1.]]


In [7]:
print(L)
print(dg.deps_no_diag)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
[(0, 1), (2, 3), (4, 5)]


In [8]:
lm = LabelModel(k=1)
lm.config['higher_order_cliques'] = False
lm.config['verbose'] = True

In [9]:
lm.train(L=dg.L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
(9, 9)
[[0.4971 0.3063 0.2819 0.224  0.3717 0.2812 0.2698 0.3063 0.188 ]
 [0.3063 0.3284 0.198  0.1791 0.29   0.2268 0.2177 0.3063 0.1492]
 [0.2819 0.198  0.4982 0.2295 0.2949 0.2134 0.2038 0.1856 0.2295]
 [0.224  0.1791 0.2295 0.2742 0.2419 0.1921 0.1835 0.1675 0.2295]
 [0.3717 0.29   0.2949 0.2419 0.5047 0.3332 0.3332 0.2705 0.2022]
 [0.2812 0.2268 0.2134 0.1921 0.3332 0.3468 0.3332 0.2109 0.1614]
 [0.2698 0.2177 0.2038 0.1835 0.3332 0.3332 0.3332 0.2023 0.1538]
 [0.3063 0.3063 0.1856 0.1675 0.2705 0.2109 0.2023 0.3063 0.1401]
 [0.188  0.1492 0.2295 0.2295 0.2022 0.1614 0.1538 0.1401 0.2295]]
Computing O^{-1}...
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
O unnorm   [4971.0  3063.0  2819.0  2240.0  3717.0  2812.0  2698.0

In [10]:
lm._set_constants(dg.L.T)
#lm._set_dependencies(dg.deps_no_diag)
mu = compute_mu(lm._get_augmented_label_matrix(dg.L.T), dg.Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.5881621 ]
 [0.44516855]
 [0.48727208]
 [0.37264612]
 [0.61286926]
 [0.46062994]
 [0.44277072]
 [0.4156368 ]
 [0.31203836]]
Average absolute error: 0.1982972010492831
Average absolute error: 0.18801862275865344


In [11]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=50000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
Using cached O_inv...
(9, 9)
[[0.4971 0.3063 0.2819 0.224  0.3717 0.2812 0.2698 0.3063 0.188 ]
 [0.3063 0.3284 0.198  0.1791 0.29   0.2268 0.2177 0.3063 0.1492]
 [0.2819 0.198  0.4982 0.2295 0.2949 0.2134 0.2038 0.1856 0.2295]
 [0.224  0.1791 0.2295 0.2742 0.2419 0.1921 0.1835 0.1675 0.2295]
 [0.3717 0.29   0.2949 0.2419 0.5047 0.3332 0.3332 0.2705 0.2022]
 [0.2812 0.2268 0.2134 0.1921 0.3332 0.3468 0.3332 0.2109 0.1614]
 [0.2698 0.2177 0.2038 0.1835 0.3332 0.3332 0.3332 0.2023 0.1538]
 [0.3063 0.3063 0.1856 0.1675 0.2705 0.2109 0.2023 0.3063 0.1401]
 [0.188  0.1492 0.2295 0.2295 0.2022 0.1614 0.1538 0.1401 0.2295]]
Estimating Z...
[Epoch 0] Loss: 274.657806
[Epoch 1000] Loss: 7.840259
[Epoch 2000] Loss: 7.840260
[Epoch 3000] Loss: 7.840260
[

In [12]:
lm._set_constants(L.T)
#lm._set_dependencies(dg.deps_no_diag)
Y = np.zeros(dg.Y.shape,dtype=float)
for i in range(n):
    if dg.Y[i] > 0:
        Y[i] = 1
    else:
        Y[i] = 0
mu = compute_mu(lm._get_augmented_label_matrix(L.T), Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.5881624 ]
 [0.44516844]
 [0.4872725 ]
 [0.3726461 ]
 [0.6128695 ]
 [0.46062997]
 [0.44277057]
 [0.41563904]
 [0.31203586]]
Average absolute error: 0.19829715469011497
Average absolute error: 0.1880185598426395


### Changing learning rate

In [13]:
lm.train(L=L.T,
         deps=dg.deps_no_diag,
         all_unary_cliques=True,
        higher_order_cliques=True,
        n_epochs=100000,
        print_every=5000,
        lr=0.0001,
        l2=0,
        O_inv_prec=1024,)

I GOT HERE
I GOT HERE 2
I GOT HERE 3
INIT 1
GET CLIQUE TREE 1
GET CLIQUE TREE 2
GET CLIQUE TREE 3
GET CLIQUE TREE 4
GET CLIQUE TREE 5
INIT 2
CLIQUE TREE 1
START INDEX:  9
I GOT HERE 4
L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
Using cached O_inv...
(9, 9)
[[0.4971 0.3063 0.2819 0.224  0.3717 0.2812 0.2698 0.3063 0.188 ]
 [0.3063 0.3284 0.198  0.1791 0.29   0.2268 0.2177 0.3063 0.1492]
 [0.2819 0.198  0.4982 0.2295 0.2949 0.2134 0.2038 0.1856 0.2295]
 [0.224  0.1791 0.2295 0.2742 0.2419 0.1921 0.1835 0.1675 0.2295]
 [0.3717 0.29   0.2949 0.2419 0.5047 0.3332 0.3332 0.2705 0.2022]
 [0.2812 0.2268 0.2134 0.1921 0.3332 0.3468 0.3332 0.2109 0.1614]
 [0.2698 0.2177 0.2038 0.1835 0.3332 0.3332 0.3332 0.2023 0.1538]
 [0.3063 0.3063 0.1856 0.1675 0.2705 0.2109 0.2023 0.3063 0.1401]
 [0.188  0.1492 0.2295 0.2295 0.2022 0.1614 0.1538 0.1401 0.2295]]
Estimating Z...
[Epoch 0] Loss: 341.925812
[Epoch 1000] Loss: 7.840259
[Epoch 2000] Loss: 7.840260
[Epoch 3000] Loss: 7.840260
[

In [19]:
lm._set_constants(L.T)
#lm._set_dependencies(dg.deps_no_diag)
Y = np.zeros(dg.Y.shape,dtype=float)
for i in range(n):
    if dg.Y[i] > 0:
        Y[i] = 1
    else:
        Y[i] = 0
mu = compute_mu(lm._get_augmented_label_matrix(L.T), Y.T, K, np.full(K, 1/K))


# Test against the true parameter values
mu_est = lm.mu.detach().numpy()
print(mu_est)
print(f"Average absolute error: {np.mean(np.abs(mu_est - mu))}")

mu_est_sm = mu_est[:6]
mu_true = dg.mu
mu_est_tr = 0.5*(mu_est_sm) + 1
print(f"Average absolute error: {np.mean(np.abs(mu_est_sm - mu_true))}")

L AUG SAHPE:  (10000, 9)
ALL UNARY CLIQUES
HIGHER ORDER CLIQUES
[[0.5881671 ]
 [0.44516578]
 [0.4872723 ]
 [0.3726459 ]
 [0.6128693 ]
 [0.46062982]
 [0.4427704 ]
 [0.415639  ]
 [0.31203815]]
Average absolute error: 0.19829678050540053
Average absolute error: 0.18801866580645243


In [15]:
from amc_utils import *
O = dg.O
O_inv = dg.Oinv
real_deps = dg.deps
mu_rec = solveMatrixCompletionWithMu(O_inv,O,real_deps)
print(f"Average absolute error: {np.mean(np.abs(mu_rec - mu_true))}")

Average absolute error: 0.0786437762845513
